<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       ModelOps demo: Model Factory Accelerator
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style ='font-size:18px;font-family:Arial;color:#00233C'><b>Introduction</b></p>

<p style='font-size:16px;font-family:Arial;color:#00233C'>The purpose of the <strong>Model Factory Solution Accelerator</strong> using ClearScape Analytics is to demonstrate a streamlined and accelerated approach to the end-to-end process of developing, deploying, and managing machine learning models within an organization, with <strong>Horizontal Scale</strong>, by operationalizing <strong>hundreds of models with minimum effort</strong>. It leverages the scalability of in-database analytics and the openness of supporting common model formats such as H2O, Dataiku and DataRobot. This unique combination enhances efficiency, scalability, and consistency across various stages of the machine learning lifecycle in Enterprise environments.</p>

<p style='font-size:16px;font-family:Arial;color:#00233C'>By incorporating best practices, automation, and standardized workflows, Model Factory enables teams to rapidly select the data to be used, configure any required models, ensure reuse, and deploy <strong>an unlimited numbers of models</strong> seamlessly into production. Ultimately, it aims to reduce the time-to-value for machine learning initiatives and promote a more structured and efficient approach to building and deploying models at scale. Here is the diagram of an automated Workflow:</p>

![image](images/12_01.png) 

<p style='font-size:16px;font-family:Arial;color:#00233C'>Similarly we can achieve this Model Factory solution using <b><u>Airflow</u></b>. The steps to be followed are mentioned<a href='https://quickstarts.teradata.com/modelops/execute-airflow-workflows-with-clearscape-analytics-modelops-model-factory-solution.html#_initialize_airflow_in_docker_compose'> here</a> in the quick starts.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Steps in this Notebook</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
 <li>Configure the Environment </li>
 <li>Parameters definition and Environment Variables</li>
 <li>Secure connection to ModelOps</li>
 <li>Model Lifecycle Workflow functions</li>
 <li>Execution of the Workflow</li>

<p style ='font-size:16px;font-family:Arial;color:#00233C'>Here are the steps to implement Model Factory Solution Accelerator using Python commands with the Restful API layer of ClearScape Analytics ModelOps. Restful API layer is all documented in the ModelOps user interface at using Swagger, it can be used to orchestrate ModelOps Agent process from external applications or schedulers such as Airflow or Gitlab. An example of using Airflow can be found at <a href="https://quickstarts.teradata.com">Teradata Getting Started</a></p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Configure the Environment</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.1 Libraries installation</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>A restart of the Kernel is needed to confirm changes</b>. We use -q parameter for a non-verbose log of the installation command, you may remove this parameter if you want to know all the steps of the pip installation.</p>

In [ ]:
#%pip install -q teradataml==17.20.0.6 teradatamodelops==7.0.3 matplotlib==3.8.2

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the mentioned versions of the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>1.2 Libraries import</b></p>

In [ ]:
import requests
import json
import time
import logging
import os
import sys
from getpass import getpass

logging.basicConfig(encoding="utf-8", level=logging.INFO)
requests.packages.urllib3.disable_warnings()


<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Parameters definition and Environment Variables</b></p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.1 ModelOps parameters</b></p>

<div class="alert alert-block alert-warning">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Important Note: </b>Replace to use your ModelOps URL. You can copy the HOST address from the Connection details on your dashboard for the url</p>
</div>

In [ ]:
# Example modelops-apr24-ver20-63z6fpyuh8lhkz39.env.clearscape.teradata.com
hostname = input("Host NAME of the machine for ModelOps:")

In [ ]:
# base domain for ModelOps
url = "https://" + hostname + "/modelops"
print(url)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> These configuration parameters include the IDs of Project, Model, Dataset Connection, Dataset training, Dataset Evaluation, Dataset template.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
These IDs have been taken from preloaded Demo project of ModelOps using XGBoost classification model. Additional preloaded models are available, you can get the Model ID by doing a right click at the Model ID from the Model catalog list.
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Additional parameters like Python Image for training and deployment are set.
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Then we are applying specific hyper parameters for the training logic, and applying resources (CPU, memory) for training, evaluation and deployment jobs.
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this case the model will be deployed as <b>batch</b> and scheduled as <b>run once</b>.</p>

In [ ]:
# secret for account "service-account-modelops-cli"
#secret = ""
# or user's password
password = getpass("Enter user's password: ")
# project id
project = "23e1df4b-b630-47a1-ab80-7ad5385fcd8d"
# model id
model = "f937b5d8-02c6-5150-80c7-1e4ff07fea31"
# dataset connection id
dataset_connection = "151abf05-1914-4d38-a90d-272d850f212c"
# dataset training id
dataset_train = "ba39e766-2fdf-426f-ba5c-4ca3e90955fc"
# dataset evaluation id
dataset_eval = "74489d62-2af5-4402-b264-715e151a420a"
# dataset template id, useful for batch deployment
dataset_template = "d8a35d98-21ce-47d0-b9f2-00d355777de1"
# Docker image for training
training_image = "artifacts.td.teradata.com/tdproduct-docker-snapshot/avmo/aoa-python-base:3.9.13-1"
# Docker image for deployment
deployment_image = "artifacts.td.teradata.com/tdproduct-docker-snapshot/avmo/aoa-python-base:3.9.13-1"
# hyperparameters for training, set to empty dictionary if none
hyper_params = {"eta": 0.2, "max_depth": 6}
# performance settings
train_cpu = "1000m"
train_memory = "1Gi"
eval_cpu = "2000m"
eval_memory = "5Gi"
deploy_memory = "3Gi"
deploy_cpu = "3000m"

deployment_type = "batch" # batch or restful
batch_type = "run_once" # publish_only or run_once . Option only for batch deployment

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.2 Additional Environment variables for deployment</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> This environment variable is needed for Resftul deployments. Variables that start with `DEPLOYVAR_` will be stripped from the prefix and passed to deployment variables.\
Example: `os.environ["DEPLOYVAR_REPLICAS"]` becomes `REPLICAS` in deployment arguments

In [ ]:
os.environ["DEPLOYVAR_REPLICAS"] = "1"
#os.environ["DEPLOYVAR_INGRESS_NAME"] = "semantic-search-ai"

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2.3 Jobs statuses categories</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> We have included following status for failure, success and running:

In [ ]:
failure_cases = ["ERROR", "CANCELLED"]
success_cases = [
    "EVALUATED",
    "TRAINED",
    "IMPORTED",
    "DEPLOYED",
    "APPROVED",
    "COMPLETED",
]
running_cases = ["CREATED", "SCHEDULED", "RUNNING", "WAITING", "ASSIGNED"]

<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Secure connection to ModelOps</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'> We are assuming Model Factory will be an automated execution, therefore we will be using a service connection for ModelOps. Here we will use the secret credential using the url and the password entered in the above step.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.1 Get a token from Keycloak through the secret credential</b></p>

In [ ]:
def get_token_from_secret(url, secret):
    data = {
        "grant_type": "client_credentials",
        "client_id": "modelops-cli",
        "client_secret": secret,
    }
    try:
        r = requests.post(
            url + "/sso/realms/teradata/protocol/openid-connect/token",
            data=data,
            verify=False,
        )
        r.raise_for_status()
    except requests.exceptions.HTTPError as e:
        logging.error(f"HTTP Error {e.args[0]}")
        sys.exit(1)
    return r.json()["access_token"]

def get_token_from_password(url, password):
    data = {
        "grant_type": "password",
        "client_id": "modelops",
        "username": "demo_user",
        "password": password,
    }
    try:
        r = requests.post(
            url + "/sso/realms/teradata/protocol/openid-connect/token",
            data=data,
            verify=False,
        )
        r.raise_for_status()
    except requests.exceptions.HTTPError as e:
        logging.error(f"HTTP Error {e.args[0]}")
        sys.exit(1)
    return r.json()["access_token"]


<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Model Lifecycle Workflow functions</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'> For each ModelOps job in the model lifecycle, we are preparing a specific function, that will take the input parameters and will do the Post into the Restful layer. This is done for Model Training, Evaluation, Approval, Deployment and Retirement

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.1 Model Training function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We take all the required input parameters, that we have already predefined in the beginning of this notebook, for the Training Job</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> The  parameters used for training job are :</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>url</li>
    <li>token</li>
    <li>model</li>
    <li>dataset_connection</li>
    <li>dataset_train</li>
    <li>train_memory</li>
    <li>train_cpu</li>
    <li>hyper_params</li>
    <li>training_image</li>    </ul></p>

In [ ]:
def train(url, token, project, model, dataset_connection, dataset_train, train_memory, train_cpu, hyper_params, training_image):
    headers = {
        "AOA-Project-ID": project,
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    data = {
        "datasetId": dataset_train,
        "datasetConnectionId": dataset_connection,
        "automationOverrides": {
                "resources": {"memory": train_memory, "cpu": train_cpu},
                "dockerImage": training_image,
            },
        "modelConfigurationOverrides": {
            "hyperParameters": hyper_params,
        },
    }
    try:
        r = requests.post(
            f"{url}/core/api/models/{model}/train",
            data=json.dumps(data),
            headers=headers,
            verify=False,
        )
        r.raise_for_status()
    except requests.exceptions.HTTPError as e:
        logging.error(f"HTTP Error {e.args[0]}")
        sys.exit(1)
    return r.json()["id"]


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.2 Model Evaluation function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> We take all the required input parameters, that we have already predefined in the beginning of this notebook, for the Evaluation Job</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> The  parameters used for evaluation job are :</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>url</li>
    <li>token</li>
    <li>project</li>
    <li>version_id</li>
    <li>dataset_connection</li>
    <li>dataset_eval</li>
    <li>eval_memory</li>
    <li>eval_cpu</li>
    <li>hyper_params</li>
    <li>deployment_image</li>    </ul></p>

In [ ]:
def evaluate(url, token, project, version_id, dataset_connection, dataset_eval, eval_memory, eval_cpu, hyper_params, deployment_image):
    headers = {
        "AOA-Project-ID": project,
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    data = {
        "datasetId": dataset_eval,
        "datasetConnectionId": dataset_connection,
        "automationOverrides": {
                "resources": {"memory": eval_memory, "cpu": eval_cpu},
                "dockerImage": deployment_image,
            },
        "modelConfigurationOverrides": {
            "hyperParameters": hyper_params,
        },
    }
    try:
        r = requests.post(
            f"{url}/core/api/trainedModels/{version_id}/evaluate",
            data=json.dumps(data),
            headers=headers,
            verify=False,
        )
        r.raise_for_status()
    except requests.exceptions.HTTPError as e:
        logging.error(f"HTTP Error {e.args[0]}")
        sys.exit(1)
    return r.json()["id"]


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.3 Model Approval function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>  We take all the required input parameters, that we have already predefined in the beginning of this notebook, for the Approval Job. If you'd like to change the comment then you can replace the "Approved" message with the message of your choice </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> The  parameters used for approval job are :</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>url</li>
    <li>token</li>
    <li>project</li>
    <li>version_id</li>
      </ul></p>

In [ ]:
def approve(url, token, project, version_id):
    from base64 import encodebytes as b6

    comment = b6(b"Approved").decode().strip()
    headers = {
        "AOA-Project-ID": project,
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    try:
        r = requests.post(
            f"{url}/core/api/trainedModels/{version_id}/approve",
            headers=headers,
            data=json.dumps({"comments": comment}),
            verify=False,
        )
        r.raise_for_status()
    except requests.exceptions.HTTPError as e:
        logging.error(f"HTTP Error {e.args[0]}")
        sys.exit(1)
    return r.json()["id"]


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.4 Model Deployment function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> Here we need to get the environment variable we defined in the beginning for number of replicas.

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Pick and transform the deployment variables</p>

In [ ]:
def get_extravars():
    extravars = {}
    for v in os.environ.keys():
        if "DEPLOYVAR_" in v:
            extravars[v.replace("DEPLOYVAR_", "")] = os.environ[v]
    return extravars


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Deployment Python function</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>  We take all the required input parameters, that we have already predefined in the beginning of this notebook, for the Deployment Job </p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> The  parameters used for deployment job are :</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>url</li>
    <li>token</li>
    <li>project</li>
    <li>version_id</li>
    <li>deploy_memory</li>
    <li>deploy_cpu</li>
    <li>dataset_connection</li>
    <li>dataset_template</li>
    <li>deployment_image</li>
       </ul></p>

In [ ]:
def deploy(url, token, project, version_id, deploy_memory, deploy_cpu, dataset_connection, dataset_template, deployment_image):
    headers = {
        "AOA-Project-ID": project,
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    extravars = get_extravars()
    data = { "engineTypeConfig": {
            "dockerImage": deployment_image,
            "resources": {"memory": deploy_memory, "cpu": deploy_cpu},
        },
            "language": "python",
            "args": extravars
    }
    
    if deployment_type == "restful":
        data["engineType"] = "DOCKER_RESTFUL"
        data["publishOnly"] = False
        data["engineTypeConfig"]["replicas"] = 1
        data["engineTypeConfig"]["engine"] = "python"
    elif deployment_type == "batch":
        data["engineType"] = "DOCKER_BATCH"
        data["datasetConnectionId"] = dataset_connection
        data["datasetTemplateId"] = dataset_template
        data["engineTypeConfig"]["engine"] = "python-batch"
        if batch_type == "publish_only":
            data["publishOnly"] = True
        elif batch_type == "run_once":
            data["publishOnly"] = False
            data["cron"] = "@once"
        else:
            logging.error("\"batch_type\" variable must be one of \"publish_only\" or \"run_once\"")
            sys.exit(1)   
    else:
        logging.error("\"deployment\" variable must be one of \"restful\" or \"batch\"")
        sys.exit(1)
        
    try:
        r = requests.post(
            f"{url}/core/api/trainedModels/{version_id}/deploy",
            data=json.dumps(data),
            headers=headers,
            verify=False,
        )
        r.raise_for_status()
    except requests.exceptions.HTTPError as e:
        logging.error(f"HTTP Error {e.args[0]}")
        sys.exit(1)
    return r.json()["id"]




<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.5 Model Retirement function</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'> We take all the required input parameters, that we have already predefined in the beginning of this notebook, for the Retirement Job. In this case we retire all the model versions that are not the same as the model version we trained in this notebook. </p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> The  parameters used for retirement job are :</p>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>url</li>
    <li>token</li>
    <li>project</li>
    <li>model</li>
    <li>version_id</li>      </ul></p> 

In [ ]:
def retire_others(url, token, project, model, version_id):
    deployments = []
    headers = {
        "AOA-Project-ID": project,
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    try:
        logging.info("Retrieving deployments")
        r = requests.get(
            f"{url}/core/api/deployments/search/findByStatus?status=DEPLOYED&projection=expandDeployment",
            headers=headers,
            verify=False,
        )
        r.raise_for_status()
        deploy_list = r.json()["_embedded"]["deployments"]
        deployments = [
            {
                "model": x["model"]["id"],
                "version": x["trainedModel"]["id"],
                "deployment": x["id"],
            }
            for x in deploy_list
        ]
    except requests.exceptions.HTTPError as e:
        logging.error(f"HTTP Error {e.args[0]}")
        sys.exit(1)
    for dep in deployments:
        if dep["model"] == model and dep["version"] != version_id:
            logging.info(f"Retiring deployment {dep['deployment']}")
            try:
                data = {"deploymentId": dep["deployment"]}
                logging.info(data)
                r = requests.post(
                    f"{url}/core/api/trainedModels/{dep['version']}/retire",
                    data=json.dumps(data),
                    headers=headers,
                    verify=False,
                )
                r.raise_for_status()
            except requests.exceptions.HTTPError as e:
                logging.error(
                    f"Problem retiring deployment {dep['deployment']}: {e.args[0]}"
                )
                sys.exit(1)
            logging.info(f"Created job ID {r.json()['id']}")


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.6 Additional functions</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Generic function to watch the job and get the model version id </p>

In [ ]:
def get_job_status(url, token, project, job_id):
    status = "RUNNING"
    headers = {
        "AOA-Project-ID": project,
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }
    while status not in success_cases + failure_cases:
        time.sleep(20)
        logging.info(f"Job in status {status}, retrying in 20 seconds")
        try:
            r = requests.get(
                f"{url}/core/api/jobs/{job_id}?projection=expandJob",
                headers=headers,
                verify=False,
            )
            r.raise_for_status()
            status = r.json()["status"]
        except requests.exceptions.HTTPError as e:
            logging.error(f"HTTP Error {e.args[0]}")
            sys.exit(1)
        if status in failure_cases:
            logging.error(f"Job failed with status {status}")
            sys.exit(1)
    logging.info("Success")
    return r.json()["metadata"]["trainedModel"]["id"]


<hr style="height:2px;border:none;background-color:#00233C;">
<p><b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Execution of the Workflow</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>When running the ModelOps Agent for the model lifecycle job, you can check the status of each job in this Notebook, or you can check the Jobs panel in the left-hand menu of the DEMO project in ModelOps. 

<img src="images/12_02.png" alt="Jobs panel" />

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use this button to Launch ModelOps and check by yourself. Use "demo_user" as your username and your password of the environment

[![image](images/launchModelOps.png)](/modelops)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Obtain the token</p>

In [ ]:
logging.info("Obtaining token")
token = get_token_from_password(url, password)

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1 Model Training execution</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Running the ModelOps Agent for training job and get the job id</p>

In [ ]:
logging.info("Training")
train_job = train(url, token, project, model, dataset_connection, dataset_train, train_memory, train_cpu, hyper_params, training_image)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Watch the job status and get model version id</p>

In [ ]:
logging.info("Getting training status")
version_id = get_job_status(url, token, project, train_job)


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2 Model Evaluation execution</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Running the ModelOps Agent for evaluation job and get the job id</p>

In [ ]:
logging.info("Evaluation")
evaluate_job = evaluate(url, token, project, version_id, dataset_connection, dataset_eval, eval_memory, eval_cpu, hyper_params, deployment_image)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Watch the job status</p>

In [ ]:
logging.info("Getting evaluation status")
version_id2 = get_job_status(url, token, project, evaluate_job)

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.3 Model Approval</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Running the ModelOps Agent for Approval job and get the job id</p>

In [ ]:
logging.info("Approval")
approve_job = approve(url, token, project, version_id)

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.3 Model Deployment execution</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Running the ModelOps Agent for Deployment job and get the job id</p>

In [ ]:
logging.info("Deployment")
deploy_job = deploy(url, token, project, version_id, deploy_memory, deploy_cpu, dataset_connection, dataset_template, deployment_image)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Watch the job status</p>

In [ ]:
logging.info("Getting deployment status")
version_id3 = get_job_status(url, token, project, deploy_job)

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2 Previous Model Retirement execution (optional)</b></p>

<div class="alert alert-block alert-warning">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Important Note: </b>The below cell should be uncommented and executed only if we want to retire the model version.</p>
</div>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Running the ModelOps Agent for retiring the deployments associated to the previous model versions in this model</p>

In [ ]:
# retirement_job = retire_others(url, token, project, model, version_id)

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            © 2024 Teradata. All rights reserved.
        </div>
    </div>
</footer>